In [1]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [2]:
client = OpenAI(base_url="http://localhost:11434/v1", api_key="EnigmaWU")

In [3]:
chat_completion = client.chat.completions.create(
  model="llama3.1",
  messages=[
    {"role":"user","content":"Hello from EnigmaWU, what's your favorite blus in the sky?"},
  ]
)

In [ ]:
print(chat_completion.choices[0].message.content)

In [5]:
from typing import Any


class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role":"system","content":system})

    def __call__(self, message):
        self.messages.append({"role":"user","content":message})
        result = self.execute()
        self.messages.append({"role":"assistant","content":result})
        return result

    def execute(self):
        chat_completion = client.chat.completions.create(
            model="llama3.1",
            messages=self.messages
        )
        return chat_completion.choices[0].message.content

In [6]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [7]:
def caculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return "ST avg 1 lbs"
    elif name in "Border Collie":
        return "BC avg 2 lbs"
    elif name in "Toy Poodle":
        return "TP avg 3 lbs"
    else:
        return "Guessing 9 lbs"
    
known_actions = {
    "calculate": caculate,
    "average_dog_weight": average_dog_weight
}

In [8]:
abot = Agent(prompt)

In [ ]:
result = abot("How much does a Bulldog weight?")
print(result)

In [10]:
result =  average_dog_weight("Bulldog")

In [ ]:
result

In [ ]:
next_prompt = "Ovservation: {}".format(result)
print(next_prompt)

In [ ]:
abot(next_prompt)

In [ ]:
abot.messages

In [15]:
abot = Agent(prompt)

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier.
what is their combined weight?"""
abot(question)

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

In [ ]:
abot(next_prompt)

In [ ]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)


In [ ]:
abot(next_prompt)

In [ ]:
next_prompt = "Observation: {}".format(eval("2 + 1"))
print(next_prompt)

In [ ]:
abot(next_prompt)


In [23]:
action_re = re.compile(r'^Action: (\w+): (.*)$')

In [24]:
def query(question, max_turns=5):
  i = 0
  bot = Agent(prompt)
  next_prompt = question
  while i < max_turns:
    i += 1
    result = bot(next_prompt)
    actions = [
      action_re.match(a)
      for a in result.split("\n")
      if action_re.match(a)
    ]
    if actions:
      action, action_input = actions[0].groups()
      if action not in known_actions:
        raise Exception("Unknown action: {}".format(action, action_input))
      print(" -- running {} {}".format(action, action_input))
      observation = known_actions[action](action_input)
      print("Observation:", observation)
      next_prompt = "Observation: {}".format(observation)
    else:
      return

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)